In [1]:
import scipy
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

from src.IterativeFitting import IterativeFitting as IF
from src.CorrFuncs import covariance_matrix, trend_est

In the Smith example, $p$ and $z$ are calculated from the initial frequencies of cases and controls. So maybe just use the crude estimates for $p$ and $z$.

In [18]:
x = np.array([2,6,11])
Nx = np.array([337,167,186,212])
M1x = 451
Lx = np.array([np.log(0.80),np.log(1.16),np.log(1.57)])
vx = np.array([0.0542,0.0563,0.0563])
a0 = 165
A0x = np.array([74,90,122])
b0 = 172
B0x = np.array([93,96,90])
B0x_sum = np.sum(B0x) + b0

Defining initial parameters of $p$ and $z$.

In [3]:
p0 = b0 / B0x_sum
z0 = B0x_sum / M1x

Defining a function that allows us to construct confidence intervals

In [4]:
# lambda V: 
def CI(L,v):
    CI = []
    for l in range(len(L)):
        CI.append((L[l] - 1.96*v[l], L[l] + 1.96*v[l]))
    return CI
ci = CI(Lx,vx)

In [5]:
A0n = (1 + (a0/b0)*Lx)/(vx - 1/a0 - 1/b0)
B0n = (1 + b0/(a0*Lx))/(vx - 1/a0 - 1/b0)

In [6]:
p1 = b0/np.sum(B0n)
z1 = np.sum(B0n)/np.sum(A0n)

In [7]:
p1

1.0214460589578545

In [8]:
z1

2.2001991001983843

Get sum of squared differences and then check to see how microsoft solver works... Also look at R code probably, it will be easier. This is just an optimization problem for the following function. We need to find b0 and a0 that optimize the function. Maybe cvxpy?

In [9]:
def sum_sq_diff(pp,zp):
    return ((p0 - pp)/p0)**2 + ((z0 - zp)/z0)**2

In [10]:
sum_sq_diff(p1,z1)

4.257258005449162

According to the R code, we want to use scipy minimize. a0 and b0 will be the parameters we optimize over. Then the function should be longer but contain something like sum_sq_diff. 

In [ ]:
f_LogLik <- function (C_Val)  {
        A0 = C_Val[1]
        B0 = C_Val[2]
        
        if (CCPR == 1 | CCPR == 3) {         # case-control or X-Sectional (diseases+exposures)
          Vextra = RR_Val$Var - 1/A0 - 1/B0  
          Est_A  <- (1 + (A0*RR_Val$RR)/B0)/Vextra  # estimate of Ai
          Est_B  <- (1 + B0/(RR_Val$RR*A0))/Vextra  # estimate of Bi
        } else if(mode2 == 0) {  # prospective, diseases
          Vextra = RR_Val$Var + 1/A0 + 1/B0   
          Est_A  <- (1 + (A0*RR_Val$RR)/B0)/Vextra  # estimate of Ai
          Est_B  <- (1 + B0/(RR_Val$RR*A0))/Vextra  # estimate of Bi
        } else     {              #   ! prospective, exposures
          Vextra = RR_Val$Var - 1/A0 + 1/B0   
          Est_A  <- (1 - (A0*RR_Val$RR)/B0)/Vextra  # estimate of Ai
          Est_B  <- (-1 + B0/(RR_Val$RR*A0))/Vextra  # estimate of Bi
        }
        SumA = A0 + sum(Est_A)
        SumB = B0 + sum(Est_B)
        
        P1 = B0/SumB   # new estimate of (P-P')/P
        F1 = (P-P1)/P
        Z1 = SumB/SumA # estimate of (Z-Z')/Z
        F2 = (Z-Z1)/Z
        RR_Vextra <<- Vextra
        RR_A <<- Est_A
        RR_B <<- Est_B
        return(F1*F1 + F2*F2)
      }

In [20]:
def f_LogLik(C_Val):
    a0x = C_Val[0]
    b0x = C_Val[1]

    Vextra = vx - 1/a0x - 1/b0x
    Est_A = (1 + (a0x/b0x)*Lx)/(Vextra)
    Est_B = (1 + b0x/(a0x*Lx))/(Vextra)
    
    SumA = a0x + np.sum(Est_A)
    SumB = b0x + np.sum(Est_B)

    p1 = b0x / SumB
    F1 = ((p0-p1)/p0)**2
    z1 = SumB/SumA
    F2 = ((z0-z1)/z0)**2
    global vx, Lx
    vx = 1/a0x - 1/b0x + 1/Est_A - 1/Est_B
    Lx = np.log(Est_A) + np.log(b0x) - np.log(a0) - np.log(Est_B)

    return F1 + F2

SyntaxError: name 'vx' is used prior to global declaration (<ipython-input-20-39b05b3ff9a4>, line 16)

In [12]:
a0_b0_res = scipy.optimize.minimize(f_LogLik,[a0,b0])

In [16]:
a0_b0_res

      fun: 3.005213586181914e-08
 hess_inv: array([[12224.87166062,  5138.89323202],
       [ 5138.89323202,  7652.06628875]])
      jac: array([ 2.47194269e-06, -2.27475125e-06])
  message: 'Optimization terminated successfully.'
     nfev: 96
      nit: 25
     njev: 32
   status: 0
  success: True
        x: array([145.37128157,  91.45251652])

In [13]:
a0, b0 = a0_b0_res.x[0], a0_b0_res.x[1]

In [14]:
a0

145.3712815719246

In [15]:
b0

91.45251651760577

In [39]:
Vextra = vx - 1/a0 - 1/b0
A = (1 + (a0/b0)*Lx)/(Vextra)
B = (1 + b0/(a0*Lx))/(Vextra)

In [40]:
A

array([17.73449828, 32.11329056, 44.61369632])

In [41]:
B

array([-49.99788201, 136.11607203,  62.22084346])